# DataVengers - AV_Mobilty_Analytics

In [13]:
import xgboost as xgb
import numpy as np
import pandas as pd

train = pd.read_csv("train_Wc8LBpr.csv")
test = pd.read_csv("test_VsU9xXK.csv")

train['Gender'] = train['Gender'].replace(to_replace = {'Male': 0, 'Female': 1}) #XGBoost works on only numeric vectors 
test['Gender'] = test['Gender'].replace(to_replace = {'Male': 0, 'Female': 1})

type_of_cab = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4}
train['Type_of_Cab'] = train['Type_of_Cab'].replace(to_replace = type_of_cab) #XGBoost works on only numeric vectors 
test['Type_of_Cab'] = test['Type_of_Cab'].replace(to_replace = type_of_cab)


confi_index = {'A': 0, 'B': 1, 'C': 2}
train['Confidence_Life_Style_Index'] = train['Confidence_Life_Style_Index'].replace(to_replace = confi_index)
test['Confidence_Life_Style_Index'] = test['Confidence_Life_Style_Index'].replace(to_replace = confi_index)

train['Surge_Pricing_Type'] = train['Surge_Pricing_Type'] - 1

X_train = train.copy()
X_test = test.copy()

from sklearn.preprocessing import LabelEncoder
print("Label Encoding...")
for f in ['Destination_Type']:
    lbl = LabelEncoder()
    lbl.fit(list(X_train[f].values) + list(X_test[f].values)) #transforming values in Destination type using label encoder 
    X_train[f] = lbl.transform(list(X_train[f].values))
    X_test[f] = lbl.transform(list(X_test[f].values))

features = np.setdiff1d(train.columns, ['Trip_ID', 'Surge_Pricing_Type']) # returns set difference of the two arrays that have been 
                                                                          #passed as the arguments. Feature extraction, all columns except first and last       
params = {"objective": "multi:softmax","booster": "gbtree", "nthread": 4, "silent": 1,
                "eta": 0.08, "max_depth": 6, "subsample": 0.9, "colsample_bytree": 0.7,
                "min_child_weight": 1, "num_class": 3,
                "seed": 2016, "tree_method": "exact"}
dtrain = xgb.DMatrix(X_train[features], X_train['Surge_Pricing_Type'], missing=np.nan)
dtest = xgb.DMatrix(X_test[features], missing=np.nan)

nrounds = 255
watchlist = [(dtrain, 'train')]
bst = xgb.train(params, dtrain, num_boost_round=nrounds, evals=watchlist, verbose_eval=20)
test_preds = bst.predict(dtest)

submit = pd.DataFrame({'Trip_ID': test['Trip_ID'], 'Surge_Pricing_Type': test_preds+1})
submit.to_csv("XGB2.csv", index=False)

Label Encoding...
[0]	train-merror:0.500015
[20]	train-merror:0.30118
[40]	train-merror:0.295195
[60]	train-merror:0.29221
[80]	train-merror:0.28902
[100]	train-merror:0.286301
[120]	train-merror:0.283833
[140]	train-merror:0.281569
[160]	train-merror:0.279587
[180]	train-merror:0.278053
[200]	train-merror:0.276184
[220]	train-merror:0.274369
[240]	train-merror:0.272911
[254]	train-merror:0.271589
